In [10]:
from fancyimpute import KNN, SimpleFill
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn import model_selection, preprocessing
import xgboost as xgb
color = sns.color_palette()

import sklearn.metrics as sklm

from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', 500)


#import data
os.chdir("/Users/hubertjenq/kaggle/sberbank russian housing")
train_df = pd.read_csv("train.csv",parse_dates=['timestamp'])
test_df = pd.read_csv("test.csv",parse_dates=['timestamp'])

#use only macroeconomic columns from the macro data
macroecon_cols = ["balance_trade", "balance_trade_growth", "eurrub", "average_provision_of_build_contract",
"micex_rgbi_tr", "micex_cbi_tr", "deposits_rate", "mortgage_value", "mortgage_rate",
"income_per_cap", "rent_price_4+room_bus", "museum_visitis_per_100_cap", "apartment_build"]
macro_df = pd.read_csv("macro.csv", parse_dates=['timestamp'], usecols=['timestamp'] + macroecon_cols)

# Data Quality Changes

#fix wrong state to the most occured state
train_df.loc[train_df['state'] == 33, 'state'] = train_df['state'].mode().iloc[0]

#fix build year errors
train_df[train_df['build_year']==0]['build_year']=np.nan
train_df[train_df['build_year']==1]['build_year']=np.nan
train_df[train_df['build_year']==3]['build_year']=np.nan
train_df[train_df['build_year']==4965]['build_year']=np.nan
train_df[train_df['build_year']==71]['build_year']=np.nan

train_df['build_year']=train_df['build_year'].replace(215,2015)
train_df['build_year']=train_df['build_year'].replace(20052009,2005)
train_df['build_year']=train_df['build_year'].replace(20,2000)

# Merge test/train to make change/make new features

#Make a data frame with all the data in it

#count test/train data
num_train=train_df.shape[0]
num_test=train_df.shape[0]

#save id and price of the train/test 
train_id=train_df['id']
train_logprice=np.log1p(train_df['price_doc'].values)
test_id=test_df['id']

#drop id/price
train_df.drop(['id','price_doc'],axis=1,inplace=True)
test_df.drop(['id'],axis=1,inplace=True)

#Combine test/train and the macro data by timestamp
all_df = pd.concat([train_df,test_df])
all_df = pd.merge_ordered(all_df,macro_df,on='timestamp',how='left')
all_df.head()


# Make new features/remove some

#Convert timestamp to year, month, day of week, and date 
all_df['yearsale']=pd.DatetimeIndex(all_df['timestamp']).year
all_df['monthsale']=pd.DatetimeIndex(all_df['timestamp']).month
all_df['dayofweeksale']=pd.DatetimeIndex(all_df['timestamp']).weekday
all_df['dateofsale']=pd.DatetimeIndex(all_df['timestamp']).day


#add houseage
all_df['houseage']=2020-all_df['build_year']

#add number of floors from top
all_df['floorsfromtop']=all_df['max_floor']-all_df['floor'].astype(float)
#relative floors
all_df['relativefloor']=all_df['floor']/all_df['max_floor'].astype(float)
#add average room size
all_df['avgroomsize']=all_df['full_sq']/all_df['num_room'].astype(float)
#add kitchen size percentage of house
all_df['kitchenpercentage']=all_df['kitch_sq']/all_df['full_sq'].astype(float)

#drop timestamp
all_df.drop(['timestamp'],inplace=True,axis=1)

all_df.head()

all_obj_df=all_df.select_dtypes(include=['object']).copy()
all_num_df=all_df.select_dtypes(exclude=['object'])


/Users/hubertjenq/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [28]:
all_num_df_subset=all_num_df

In [12]:
all_num_df_subset.shape

(100, 296)

In [29]:
all_num_df_subset_knn=KNN(k=3).complete(all_num_df_subset)

Imputing row 1/38133 with 14 missing, elapsed time: 2285.887
Imputing row 101/38133 with 11 missing, elapsed time: 2286.635
Imputing row 201/38133 with 12 missing, elapsed time: 2287.250
Imputing row 301/38133 with 12 missing, elapsed time: 2287.706
Imputing row 401/38133 with 12 missing, elapsed time: 2288.176
Imputing row 501/38133 with 12 missing, elapsed time: 2288.686
Imputing row 601/38133 with 12 missing, elapsed time: 2289.201
Imputing row 701/38133 with 11 missing, elapsed time: 2289.660
Imputing row 801/38133 with 12 missing, elapsed time: 2290.103
Imputing row 901/38133 with 14 missing, elapsed time: 2290.588
Imputing row 1001/38133 with 14 missing, elapsed time: 2291.073
Imputing row 1101/38133 with 11 missing, elapsed time: 2291.506
Imputing row 1201/38133 with 11 missing, elapsed time: 2291.963
Imputing row 1301/38133 with 15 missing, elapsed time: 2292.381
Imputing row 1401/38133 with 14 missing, elapsed time: 2292.815
Imputing row 1501/38133 with 11 missing, elapsed tim

Imputing row 13001/38133 with 1 missing, elapsed time: 2344.264
Imputing row 13101/38133 with 0 missing, elapsed time: 2344.517
Imputing row 13201/38133 with 0 missing, elapsed time: 2344.729
Imputing row 13301/38133 with 3 missing, elapsed time: 2344.912
Imputing row 13401/38133 with 10 missing, elapsed time: 2345.129
Imputing row 13501/38133 with 2 missing, elapsed time: 2345.349
Imputing row 13601/38133 with 8 missing, elapsed time: 2345.549
Imputing row 13701/38133 with 4 missing, elapsed time: 2345.839
Imputing row 13801/38133 with 1 missing, elapsed time: 2346.266
Imputing row 13901/38133 with 8 missing, elapsed time: 2346.579
Imputing row 14001/38133 with 2 missing, elapsed time: 2346.914
Imputing row 14101/38133 with 10 missing, elapsed time: 2347.231
Imputing row 14201/38133 with 3 missing, elapsed time: 2347.551
Imputing row 14301/38133 with 13 missing, elapsed time: 2347.849
Imputing row 14401/38133 with 1 missing, elapsed time: 2348.153
Imputing row 14501/38133 with 2 missi

Imputing row 25801/38133 with 1 missing, elapsed time: 2380.045
Imputing row 25901/38133 with 5 missing, elapsed time: 2380.387
Imputing row 26001/38133 with 7 missing, elapsed time: 2380.722
Imputing row 26101/38133 with 20 missing, elapsed time: 2381.052
Imputing row 26201/38133 with 23 missing, elapsed time: 2381.308
Imputing row 26301/38133 with 1 missing, elapsed time: 2381.569
Imputing row 26401/38133 with 1 missing, elapsed time: 2381.858
Imputing row 26501/38133 with 1 missing, elapsed time: 2382.145
Imputing row 26601/38133 with 16 missing, elapsed time: 2382.398
Imputing row 26701/38133 with 2 missing, elapsed time: 2382.661
Imputing row 26801/38133 with 2 missing, elapsed time: 2382.897
Imputing row 26901/38133 with 1 missing, elapsed time: 2383.153
Imputing row 27001/38133 with 1 missing, elapsed time: 2383.512
Imputing row 27101/38133 with 1 missing, elapsed time: 2383.747
Imputing row 27201/38133 with 29 missing, elapsed time: 2384.045
Imputing row 27301/38133 with 8 miss

In [32]:
knnimpute.to_csv('knnxvalues.csv', index=False)

In [31]:
knnimpute=pd.DataFrame(all_num_df_subset_knn)

In [ ]:
#Change catagorical variables to factors

for o in all_obj_df:
    all_obj_df[o] = pd.factorize(all_obj_df[o])[0]

#put categorical and numeric variables back together
all_df_values=pd.concat([all_num_df,all_obj_df],axis=1)

#convert to numeric values
all_x = all_df_values.values

colnames=all_df.columns

#split all data back into test/training

train_x = all_x[:num_train]
test_x = all_x[num_train:]

#choose 7626 training and 30507 test
indices = np.random.permutation(train_x.shape[0])
valind = indices[:7626]
trainind = indices[7626:]

val_x = train_x[valind]
trainmod_x = train_x[trainind]
val_y = train_logprice[valind]
trainmod_y = train_logprice[trainind]
